In [1]:
import random
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import warnings
from tqdm import tqdm_notebook as tqdm
import requests
import json
import API
import matplotlib.pyplot as plt
import GoogleDistanceMatrix
import GoogleVRP
import routing
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
warnings.filterwarnings('ignore')

In [3]:
order = pd.read_excel('../1. Sales Data/DMS order_2-30Nov2020 (Only store with DMS available)+Transportation.xlsx')
order = order.loc[order['Instant Order']==True]
store = pd.read_csv('Data/StoreLocation.csv')

In [5]:
order['orderTime'] = pd.to_datetime(order['下單時間'])
order['arrivalTime'] = pd.to_datetime(order['訂單送達時間'])
order['departureTime'] = pd.to_datetime(order['取餐出發時間'])

order['waitingTime'] = order['arrivalTime'] - order['orderTime']
order['tripDurationReal'] = pd.to_datetime(order['訂單送達時間']) - pd.to_datetime(order['departureTime'])
order['tripDurationReal'] = order['tripDurationReal'].dt.seconds

In [6]:
def get_lat_lon(x):
    location = x['配送地址']
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=香港'+location+'&key='+API.key
    data = requests.get(url)
    text = json.loads(data.text)
    if text['results'] == []:
        return('missing')
    lat = str(text['results'][0]['geometry']['location']['lat'])
    lng = str(text['results'][0]['geometry']['location']['lng'])
    return(lat+','+lng)

In [ ]:
order['DeliveryLatLon'] = order.apply(get_lat_lon,axis=1)

In [ ]:

def tripDurationFromGoogle(x):
    # distance matrix API
    StoreLatLon,DeliveryLatLon = x['StoreLatLon'],x['DeliveryLatLon']
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins='+StoreLatLon+\
    '&destinations='+DeliveryLatLon+'&key='+API.key
    data = requests.get(url)
    text = json.loads(data.text)
    if text['rows'] != []:
        if 'duration' in text['rows'][0]['elements'][0]:
            
            duration = text['rows'][0]['elements'][0]['duration']['value']
            return duration
    else:
        return 'missing'

In [ ]:
order = order[['日期', '門店名稱', '配送地址', '下單時間', '指派成功時間', '騎手接單時間', '到店取餐時間', '取餐出發時間',
       '訂單送達時間', '預送達時間', 'Banner', 'Instant Order', '收貨地址坐標', 'Rider',
       'Transportation', 'orderTime', 'arrivalTime', 'waitingTime',
       'last取餐出發時間', 'lastRider', 'departureTime', 'tripDurationReal',
       'DeliveryLatLon','shared']]

In [ ]:
order = order.merge(store, how='left',left_on='門店名稱', right_on='storeCode')
order['StoreLatLon'] = order['lat'].astype(str)+','+order['lon'].astype(str)

In [ ]:
order['tripDurationFromGoogle'] = order.apply(lambda x:tripDurationFromGoogle(x),axis=1)

In [ ]:
order['tripDurationFromGoogle'].fillna('missing',inplace=True)
order = order.loc[order['tripDurationFromGoogle']!='missing']
order['tripDurationFromGoogle'] = order['tripDurationFromGoogle'].astype(int)

In [ ]:
order.loc[order['shared']==0]['tripDurationFromGoogle'].mean()/60

In [ ]:
order.loc[order['shared']==0]['tripDurationReal'].mean()/60

In [ ]:
timeParameter = (order['tripDurationReal']/60).mean() / (order['tripDurationFromGoogle']/60).mean()
timeParameter

In [ ]:
order['foodPreparationTime'] = pd.to_datetime(order['取餐出發時間']) - pd.to_datetime(order['下單時間'])

In [ ]:
order.loc[order['shared']==0]['foodPreparationTime'].mean()

In [ ]:
order.loc[order['shared']==1]['foodPreparationTime'].mean()

In [ ]:
order.to_csv('Data/ProcessedNov30.csv',index=False)